In [2]:
#https://teddylee777.github.io/tensorflow/news-sarcasm
!pip install tensorflow==2.1.0
import json
import tensorflow as tf
import numpy as np
import urllib

from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

url = 'https://storage.googleapis.com/download.tensorflow.org/data/sarcasm.json'
urllib.request.urlretrieve(url, 'sarcasm.json')

vocab_size = 1000
embedding_dim = 16
max_length = 120
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"
training_size = 20000
batch_size = 256
sentences = []
labels = []

with open('sarcasm.json', 'r') as f:
    datastore = json.load(f)

for item in datastore:
    sentences.append(item['headline'])
    labels.append(item['is_sarcastic'])


tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok) #빈도수가 높은 단어 top 1000개 
tokenizer.fit_on_texts(sentences)

sequences = tokenizer.texts_to_sequences(sentences) #텍스트 > 숫자 

padded = pad_sequences(sequences, truncating=trunc_type, padding=padding_type, maxlen=max_length) #(,120)으로 설정

#train, valid 분리
train_padded = padded[:training_size]
train_labels = labels[:training_size]

validation_padded = padded[training_size:]
validation_labels = labels[training_size:]

train_labels = np.array(train_labels)
validation_labels = np.array(validation_labels)

model = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=max_length), #16차원으로축소
    Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
    Bidirectional(tf.keras.layers.LSTM(64)),
    Flatten(),
    Dense(16, activation='relu'),
    Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
                loss='binary_crossentropy', 
                metrics=['accuracy'])

checkpoint_path = 'my_checkpoint.ckpt'
sarcasm_checkpoint = ModelCheckpoint(checkpoint_path, 
                                                save_weights_only=True, 
                                                save_best_only=True, 
                                                monitor='val_loss',
                                                verbose=1)



hist = model.fit(train_padded, train_labels, 
            validation_data=(validation_padded, validation_labels),
            callbacks=[sarcasm_checkpoint],batch_size=batch_size,
            epochs=10)



     |████████████████████████████████| 421.8MB 27kB/s 
     |████████████████████████████████| 3.9MB 48.1MB/s 
     |████████████████████████████████| 450kB 40.1MB/s 
     |████████████████████████████████| 51kB 7.1MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7542 sha256=65e5b1b98ad949908ec86a6ac1944f99c9eef2f6491606136ce0c7da67eeb322
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.11.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorboard 2.3.0
    Uninstalling tensorboard-2.3.0:
      Successfully uninstalled tensorboard-2.3.0
  Found existing installation: tensorflow-estimator 2.3.0
    Uninstalling tensorflow-estimator-2.3.0:
      Successfully uninstalled tensorflow-estimator-2.3.0
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully 